# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Feb 17th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [1]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd

# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd
/data/pt_02586/sub_01/preprocessing


: 1

In [2]:
# We use from now on and forever the NEW bvec
# that were created by the EDDY command
# because they contain the small rotation
# that were introduced by motion correction
BVEC=$SUBJECTDIR'preprocessing/dwi_eddy.eddy_rotated_bvecs';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';

# We use from now on and forever the NEW bvec
# that were created by the EDDY command
# because they contain the small rotation
# that were introduced by motion correction
BVEC=$SUBJECTDIR'preprocessing/dwi_eddy.eddy_rotated_bvecs';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';


: 1

### Image Registration with ANTs

We use [ANTs (Advanced Normalization Tools)](http://stnava.github.io/ANTs/) for image registration.   

In this case, we want to register the FA of our individual subject to an FA atlas in MNI space.  

We will use the SyN registration from ANTs, using the command antsRegistrationSyNQuick.sh

#### Basics of image registration.

There is 3 broad "types" of registration (sometimes refered to as degrees-of-freedom (dof)):  
- **Rigid**: only translations and rotation, data is a "rigid" 3D block that gets moved around, (6 dof)  
- **Affine**: same as Rigid + uniform stretching, squeezing and shearing (sometimes called linear, 12 dof)  
- **"Nonlinear"**: Everything else, typically involving complex local deformations.

In the case of non-linear registration with ANTs, we first apply a rigid registration to match the images centers and orientation, then an affine registration to match the "scale/size" of the image and finally, a SyN registration, which is a specific nonlinear deformation algorithm.  


**Moving image**: This is the image that your are registering **TO** something, the one that gets deformed.  
**Fixed image**: This is the image that serves as target.  
**Warped image**: This is the result of the registration. Its a modified version of the moving image **IN** the space of the fixed image.  
**invWarped image**: You can also compute the opposite, i.e. a modified version of the fixed image in the moving space.

This is only practical if you have a nice symmetric type of nonlinear registration, one where you can apply the inverse transform. SyN is one such "well behave" method.  

Typically, the Warped image will also have the resolution of the fixed space.  
And the invWarped image will have the resolution of the moving space.  
For this reason, it is common to register the lower resolution image **TO** the lower resolution one.  

One important **exception** is when we have some anatomical image like a T1 and some diffusion data. In that case, you register the anatomical image to the diffusion space. This is because you cannot apply nonlinear registration to diffusion data without messing up the relation between the images and diffusion b-vectors and you therefore **HAVE** to stay in diffusion native space until you are done with your diffusion local modeling.


To recap, in our case here, the moving image is the subject's FA, the fixed image is the MNI template. Therefore, the warped image is our subject warped in MNI space and the invWarped is a deformed template in subject space. 

An finetuned usage of ANTs can be quite tricky.  
It involve manually setting the parameters of the rigid then the affine and finally the SyN registration.  
Choosing for each the number of multi-resolution level, of iteration, the similarity metric, etc.  

For example:  
```
 antsRegistration -d 3 \
                  --float 1 \
                  --verbose 1 \
                  -u 1 \
                  -w [ 0.01,0.99 ] \
                  -z 1 \
                  -r [ $fixed,$moving] \
                  -t Rigid[ 0.1 ] \
                  -m MI[$fixed,$moving,1,32,Regular,0.25 ] \
                  -c [ 1000x500x250x0,1e-6,10 ] \
                  -f 6x4x2x1 \
                  -s 4x2x1x0 \
                  -t Affine[ 0.1 ] \
                  -m MI[$fixed,$moving,1,32,Regular,0.25 ] \
                  -c [ 1000x500x250x0,1e-6,10 ] \
                  -f 6x4x2x1 \
                  -s 4x2x1x0 \
                  -t SyN[ 0.1,3,0 ] \
                  -m CC[$fixed,$moving,1,4 ] \
                  -c [ 100x100x70x20,1e-9,10 ] \
                  -f 6x4x2x1 \
                  -s 3x2x1x0 \
                  -o $output
```

However, we can now use the much more pratical script antsRegistrationSyNQuick.sh  
Where the same command would look something like:  

```
antsRegistrationSyNQuick.sh -d 3 \
    -p f \
    -f $fixed \
    -m $moving \
    -o $output \
```

#### Parameters

We need to specify the fixed (-f) and moving (-m) image, an output prefix name (-o) and specify 3D registration (-d).  

The only other relevant option for us is the number of cpu for parallelisation (-n).  

We leave the type of transform (-t) to the default for rigid+affine+SyN.  
We are using already masked data instead of a mask.  

#### outputs

This tools will ouput 5 files (all starting with the output prefix (-o)):  
- 0GenericAffine.mat: This is a matrix containing the rigid+affine parameter of the registration from moving to fixed.  
- 1Warp.nii.gz: This is a warp-image containing the nonlinear part of the registration from moving to fixed.  
- Warped.nii.gz: This is the warped image, i.e. the moving image deformed into the fixed space.  
- 1InverseWarp.nii.gz: This is another warp-image containing the inverse of the nonlinear part of the registration, i.e. it is a warp-image to bring fixed to moving.  
- InverseWarped.nii.gz: This is the invWarped image, i.e. the fixed image deformed into the moving space.  



In [3]:
antsRegistrationSyNQuick.sh -h

antsRegistrationSyNQuick.sh -h

Usage:

antsRegistrationSyNQuick.sh -d ImageDimension -f FixedImage -m MovingImage -o OutputPrefix

Example Case:

antsRegistrationSyNQuick.sh -d 3 -f fixedImage.nii.gz -m movingImage.nii.gz -o output

Compulsory arguments:

     -d:  ImageDimension: 2 or 3 (for 2 or 3 dimensional registration of single volume)

     -f:  Fixed image(s) or source image(s) or reference image(s)

     -m:  Moving image(s) or target image(s)

     -o:  OutputPrefix: A prefix that is prepended to all output files.

Optional arguments:

     -n:  Number of threads (default = ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS if defined, otherwise 1)

     -i:  initial transform(s) --- order specified on the command line matters

     -t:  transform type (default = 's')
        t: translation (1 stage)
        r: rigid (1 stage)
        a: rigid + affine (2 stages)
        s: rigid + affine + deformable syn (3 stages)
        sr: rigid + deformable syn (2 stages)
        so: deformable syn 

: 1

In [4]:
# In pratice, we have the MNI template
MNIFA='/afs/cbs.mpg.de/software/fsl/6.0.3/ubuntu-bionic-amd64/data/standard/FSL_HCP1065_FA_1mm.nii.gz'

# In pratice, we have the MNI template
ta/standard/FSL_HCP1065_FA_1mm.nii.gz'0.3/ubuntu-bionic-amd64/dat


: 1

In [ ]:
# Simple 3D registration with the MNI FA template as fixed,
# our previously created masked subject FA as the moving
# some output name
# and 20 cpu for faster processing.
antsRegistrationSyNQuick.sh -d 3 \
    -f $MNIFA \
    -m $MASKEDFAANTS \
    -o fa_eddy \
    -n 20;

# Which will output:
# part 1 of the transform "fa_eddy0GenericAffine.mat"
# part 2 of the transform "fa_eddy1Warp.nii.gz"
# part 2 of the inverse transform "fa_eddy1InverseWarp.nii.gz"
# subject FA in MNIspace "fa_eddyWarped.nii.gz"
# template in subject space "fa_eddyInverseWarped.nii.gz"

# Note that there is no InverseGenericAffine.mat because the inverse of an affine matrix is simply the inverse matrix.
# The tools handle this by themself with and "inverse affine" option

In our case, we don't actually care about the deformed subject image in template space, we actually only wanted the inverse transform whcih we can apply to anything that "lives" in MNI space and bring it into subject specific space.  

In our case, we have seeding coordinates in MNI space that were used to create rough seeding ROI in MNI space whcih can be "imported" into subject space by properly applying the inverse transform with antsApplyTransforms

In [5]:
antsApplyTransforms -h

antsApplyTransforms -h

COMMAND: 
     antsApplyTransforms

OPTIONS: 
     -d, --dimensionality 2/3/4
     -e, --input-image-type 0/1/2/3/4 
                            scalar/vector/tensor/time-series/multichannel 
     -i, --input inputFileName
     -r, --reference-image imageFileName
     -o, --output warpedOutputFileName
                  [warpedOutputFileName or compositeDisplacementField,<printOutCompositeWarpFile=0>]
                  Linear[genericAffineTransformFile,<calculateInverse=0>]
     -n, --interpolation Linear
                         NearestNeighbor
                         MultiLabel[<sigma=imageSpacing>,<alpha=4.0>]
                         Gaussian[<sigma=imageSpacing>,<alpha=1.0>]
                         BSpline[<order=3>]
                         CosineWindowedSinc
                         WelchWindowedSinc
                         HammingWindowedSinc
                         LanczosWindowedSinc
                         GenericLabel[<interpolator=Linear>]
     

: 1

In [ ]:
# using antsApplyTransforms with the result of the previous registration
# to bring some ROI image from MNI space to subject space
# -i: the ROI image to be warped
# -r: A reference image, something which is already in subject space
# it will also define the resolution of the warped image
# -t: a transform to be applied
# !!!! the order of the transforms matter,
# it is a stack, therefore it applies the transforms in reverse of reading order.
# For the case here, it first apply the second -t which is the InverseWarp
# Then it apply the inverse of the affine (notice the "1" next to the affine, indicating to useInverse)
# -o: output name for the transformed image
antsApplyTransforms -d 3 \
    -i $ROOTDIR'ROI_MNI/ifgop_peak_dilx2.nii.gz' \
    -r $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz' \
    -t [$SUBJECTDIR'preprocessing/fa_eddy0GenericAffine.mat', 1] \
    -t $SUBJECTDIR'preprocessing/fa_eddy1InverseWarp.nii.gz' \
    -o $SUBJECTDIR'preprocessing/ifgop_peak_dilx2_warped.nii.gz';


# If we were instead trying to tranform some other subject map into MNI space, 
# we would first put -t 1Warp.nii.gz and then -t 0GenericAffine.mat


### We'll discuss the specific of the MNI ROI creation when dealing with tractography next week.

### Diffusion Local Modeling

- The diffusion signal intensity is inversely proportional to the "amount of diffusion" in a given direction.  
- In our protocol, We have a lot of diffusion directions to "sample" this signal on the sphere.  

This signal on the sphere can be transformed into a **diffusion orientation distribution function (dODF)**, typically represented as a deformed sphere which represent the "amount of diffusion" at all orientations.  

- There is more diffusion along the orientation of WM bundles then across because it is easier (less obstacles) for water molecule to move in that directions.  
- Therefore, the maximas of the dODF should match bundle orienation.  
- However, there is still some diffusion in other orientations.  

This is why we want to create a **fiber orientation distribution function (fODF or FOD)** which is an object representing the "probabilities" or "proportion" of fiber at all orientation instead.  

This is very similar to the dODF but with sharper peak and it is therefore the prefered object for tractography.  

The idea is to estimate what the **dODF** looks like for voxel with only a single very well alligned bundle (no crossing, no fanning).  
We then use that special single bundle dODF (called response function) to **deconvolve** all the dODF in the brain and therefore compute de fODF. You can think of decovolution as a de-blurring on the sphere.  

We use "dwi2response" and "dwi2fod" from MRTRIX to estimate the response function and compute the fODF. 
This model is called constrained spherical deconvolution (CSD).  

And important detail of CSD is that it first fits a spherical harmonics (SH) to represent the signal, instead of working directly on the discrete bvec samples. We have to set a maximum order for the SH basis (-lmax) which will depend on the number of bvecs we have.  
The most common choice is lmax=8 (or sh order 8) but this requires 45+ bvecs to be well defined (i.e. the SH basis of order maximum 8 has 45 degree-of-freedom).  
Here we have 30 bvecs, we will therefore use lmax=6 (28 dof).
The generic formula for the number of dof of SH basis lmax is $$\frac{(l_\max+1)(l_\max+2)}{2}$$  

But in practice, you wouldnt want to use more than lmax=8 even if you have tons bvecs and you wouldnt be using CSD if you have less than 30 bvecs, so it's pretty much always order 8 or 6.  


In [ ]:
# command to estimate the single-fiber response function
# we use the fa method here which is simple and works well for in-vivo single bvalue
# This command will find the highest FA voxel inside a mask, align them and fit the SH.
# parameters:
# diffusion data
# file name to store the response function
# -fslgrad: bvecs and bvals
# -mask: (we dont want to include "artefact" voxels outside the brain as potential candidate)
# Here the mask is a tight brain mask with an FA threshold
# -voxels: file name to save a mask of which voxels where selected as single fiber population
# -lmax: SH basis order for the response function, this should always be 4
# -ntheads: number of cpu for faster computation
# -number: the number of single fiber voxel to use
dwi2response fa \
    dwi_eddy.nii.gz \
    response_fa.txt \
    -fslgrad $BVEC $BVAL \
    -mask fa_eddy_th0p3.nii.gz \
    -voxels response_voxel_fa.nii.gz \
    -lmax 4 \
    -nthreads 20 \
    -number 300;


In [ ]:
# You can visually inspect the response function with the following command:
shview response_fa.txt
# It should look like a puffy disk

In [ ]:
# command to compute csd on the brain
# we specify csd as the model since we are using single bvalue data
# parameters: 
# diffusion data
# the response function
# file name to save the FOD (it is saved as SH coefficients)
# -fslgrad: bvecs and bvals
# -mask: Here I am using a larger mask than for the response function (a BET mask)
# This is because I want to make sure I have an FOD everywhere, including low FA voxels
# (such has crossing or near cortex sometimes)
# -ntheads: number of cpu for faster computation
# -lmax: SH basis discused before, this depends on the number of bvec
dwi2fod csd \
    dwi_eddy.nii.gz \
    response_fa.txt \
    fod_lmax6.nii.gz \
    -fslgrad $BVEC $BVAL \
    -mask b0_eddy_avg_N4x3_bet_0p4_mask.nii.gz \
    -nthreads 20 \
    -lmax 6;
